In [ ]:
import math
import pandas as pd
import numpy as np
from nipype.interfaces import afni, fsl
import nibabel as nib
import matplotlib.pyplot as plt
import os, datetime
import sys
import shutil
import glob
import subprocess
from pathlib import Path
import re
import getpass
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser
from nilearn import image, plotting
import scipy.stats
from statsmodels.stats.multitest import multipletests
from matplotlib.gridspec import GridSpec
import sqlite3
from tabulate import tabulate
from io import StringIO
import getpass

#Prime functions, can be choosen to run in the next cell
def smooth_movavg(in_file, out_file, win_sec_duration, tr):

  win = max(1, int(round(win_sec_duration / tr)))
  
  # Apply moving average
  def moving_average_1d(x, win):
      k = np.ones(win, dtype=float) / win
      xpad = np.pad(x, (win//2, win-1-win//2), mode='edge')  # reduce edge shrinkage
      return np.convolve(xpad, k, mode='valid')

  img = nib.load(in_file)
  data = img.get_fdata()   # X,Y,Z,T
  T = data.shape[-1]
  flat = data.reshape(-1, T)
  sm = np.vstack([moving_average_1d(ts, win) for ts in flat]).reshape(data.shape)

  nib.Nifti1Image(sm, img.affine, img.header).to_filename(out_file)
  print(f"Wrote: {out_file}  (tr={tr}s, window={win_sec_duration}s => {win} vols)")
def bruker_to_nifti(in_path, scan_number, out_file):

    import os, glob, shutil, subprocess
    from nipype.interfaces import afni

    scan_dir = os.path.join(in_path, scan_number)
    method_file = os.path.join(scan_dir, "method")

    # -------------------------------------------------------
    # Helper: safely collect only produced NIfTI files
    # -------------------------------------------------------
    def get_nifti_files(scan_number):
        return [
            f for f in glob.glob(f"*{scan_number}*.nii*")
            if os.path.isfile(f)
        ]

    # ---------- 1) Run brkraw tonii ----------
    cmd = ["brkraw", "tonii", f"{in_path}/", "-s", str(scan_number)]
    subprocess.run(cmd, check=True)

    # ---------- 2) Detect echo count ----------
    NoOfEchoImages = None
    if os.path.exists(method_file):
        with open(method_file) as f:
            for line in f:
                if "PVM_NEchoImages=" in line:
                    echo_str = line.split("=")[1].strip()
                    try:
                        NoOfEchoImages = int(echo_str)
                    except:
                        NoOfEchoImages = 1
                    break

    # ---------- 3) Collect produced NIfTI files ----------
    src_files = get_nifti_files(scan_number)

    if not src_files:
        raise RuntimeError(
            f"No NIfTI files found after brkraw conversion for scan {scan_number}"
        )

    # ---------- 4) Single echo OR unknown echo ----------
    if NoOfEchoImages is None or NoOfEchoImages == 1:
        # Just copy first detected nifti
        shutil.copy(src_files[0], "G1_cp.nii.gz")

    # ---------- 5) Multi-echo ----------
    else:
        merged_file = f"{scan_number}_combined_images.nii.gz"

        # Merge all echoes
        subprocess.run(
            ["fslmerge", "-t", merged_file] + src_files,
            check=True
        )

        shutil.copy(merged_file, "G1_cp.nii.gz")

    # ---------- 6) Fix orientation to LPI ----------
    print(f"{bcolors.NOTIFICATION}Fixing orientation to LPI{bcolors.ENDC}")

    resample = afni.Resample()
    resample.inputs.in_file = "G1_cp.nii.gz"
    resample.inputs.out_file = out_file
    resample.inputs.orientation = "LPI"
    resample.run()

    # ---------- 7) Save NIfTI header info ----------
    with open("NIFTI_file_header_info.txt", "w") as out:
        subprocess.run(["fslhd", out_file], stdout=out, check=True)

    print_statement(
        f"[OK] Bruker → NIFTI workflow completed.",
        bcolors.OKGREEN
    )
def extract_middle_volume(in_file, reference_vol, out_file, size):
  extract_vol = fsl.ExtractROI()
  extract_vol.inputs.in_file=in_file 
  extract_vol.inputs.t_min=reference_vol 
  extract_vol.inputs.t_size=size 
  extract_vol.inputs.roi_file=out_file
  extract_vol.run()

  print("[OK] Intended Volumes extracted.")
  return out_file
def motion_correction(reference_vol, input_vol, output_prefix):

    # ---------- 1) 3dvolreg ----------
    
    volreg = afni.Volreg()  
    volreg.inputs.in_file = input_vol
    volreg.inputs.basefile = reference_vol
    volreg.inputs.out_file = f"{output_prefix}.nii.gz"
    volreg.inputs.oned_file = "motion.1D"
    volreg.inputs.args = '-linear'
    volreg.inputs.oned_matrix_save = "mats"
    volreg.inputs.oned_matrix_save = "rmsabs.1D"
    volreg.inputs.verbose = True
    volreg.run()

    print("[INFO] Running 3dvolreg…")
    return output_prefix
def plot_motion_parameters(input_file):

    # ---------- 4) Plot motion parameters ----------
    print("[INFO] Creating motion plots…")

    # Translation plots
    data = np.loadtxt(input_file)

    # If your file HAS a header, use:
    # data = np.loadtxt("your_file.1D", comments="#")

    # X-axis (row index / timepoints)
    x = np.arange(data.shape[0])

    # -------- Plot 1: first 3 columns --------
    plt.figure(figsize=(8, 4))
    for i in range(3):
        plt.plot(x, data[:, i], label=f"Column {i+1}")

    plt.title("Rotation")
    plt.xlabel("Volume Number")
    plt.ylabel("Rotation in degrees")
    plt.legend(["Pitch (x)", "Roll (y)", "Yaw (z)"])
    plt.tight_layout()
    plt.savefig("motion_rotations.svg", dpi=1200)
    # -------- Plot 2: next 3 columns --------
    plt.figure(figsize=(8, 4))
    for i in range(3, 6):
        plt.plot(x, data[:, i], label=f"Column {i+1}")

    plt.title("Translation")
    plt.xlabel("Volume Number")
    plt.ylabel("Translation in mm")
    plt.legend(["Read (x)", "Phase (y)", "Slice (z)"])
    plt.tight_layout()
    plt.savefig("motion_translations.svg", dpi=1200)
def compute_mean_range(input_file, prefix, start_idx, end_idx):
    
    afni_cmd = ["3dTstat", "-mean", "-prefix", prefix, f"{input_file}[{start_idx}..{end_idx}]"]

    print("[INFO] Running:", " ".join(afni_cmd))

    subprocess.run(afni_cmd, check=True)
    print_statement("[OK] Mean baseline image saved.", bcolors.OKGREEN)
def masking_file(input_file, mask_file, output_file):
    
    math = fsl.maths.ApplyMask()
    math.inputs.in_file = input_file
    math.inputs.mask_file = mask_file
    math.inputs.out_file = output_file

    math.run()

    print_statement(f"[OK] Masked file saved → {output_file}", bcolors.OKGREEN)
    return output_file
def tSNR(input_file, output_file, reference_vol, size):
  
  extract_middle_volume(input_file, reference_vol, "extracted_ts.nii.gz", size)

  mean = fsl.maths.MeanImage()
  mean.inputs.in_file = "extracted_ts.nii.gz"
  mean.inputs.out_file = "mean_image.nii.gz"
  mean.run()

  std = fsl.maths.StdImage()
  std.inputs.in_file = "extracted_ts.nii.gz"
  std.inputs.out_file = "std_image.nii.gz"
  std.run()

  tSNR = fsl.maths.BinaryMaths()
  tSNR.inputs.in_file = "mean_image.nii.gz"
  tSNR.inputs.operand_file = "std_image.nii.gz"
  tSNR.inputs.operation = "div"
  tSNR.inputs.out_file = output_file
  tSNR.run()

  print_statement(f"[OK] tSNR file saved → {output_file}", bcolors.OKGREEN)
  return output_file
def spatial_smoothing(input_file, output_file, fwhm):
    cmd = ["fslmaths", input_file, "-kernel", "gauss", str(fwhm), "-fmean", output_file]

    result = subprocess.run(cmd, check=True)
    print("Smoothing completed successfully.")
def signal_change_map(signal_file, baseline_file, output_file):
   
  tmp_sub = "tmp_signal_minus_baseline.nii.gz"
  tmp_div = "tmp_psc_raw.nii.gz"

  sub = fsl.BinaryMaths()
  sub.inputs.in_file = signal_file
  sub.inputs.operand_file = baseline_file
  sub.inputs.operation = "sub"
  sub.inputs.out_file = tmp_sub
  sub.run()

  div = fsl.BinaryMaths()
  div.inputs.in_file = tmp_sub
  div.inputs.operand_file = baseline_file
  div.inputs.operation = "div"
  div.inputs.out_file = tmp_div
  div.run()

  mul = fsl.BinaryMaths()
  mul.inputs.in_file = tmp_div
  mul.inputs.operation = "mul"
  mul.inputs.operand_value = 100
  mul.inputs.out_file = output_file
  mul.run()

  os.remove(tmp_sub)
  os.remove(tmp_div)

  print_statement(f"[OK] Percent Signal Change Map saved → {output_file}", bcolors.OKGREEN)
  return output_file
def coregistration_afni(input_file1=None, input_file2=None, reference_file=None, output_file1=None, output_file2=None, estimate_affine=True, apply_affine=True, affine_mat="mean_func_struct_aligned.aff12.1D"):

  results = {}

  # -------- STEP 1: Estimate affine --------
  if estimate_affine:
      if output_file1 is None:
          raise ValueError("output_file1 must be provided when estimate_affine=True")
      if input_file1 is None:
          raise ValueError("input_file1 must be provided when estimate_affine=True")

      coreg_wo_affine = afni.Allineate()
      coreg_wo_affine.inputs.in_file = input_file1
      coreg_wo_affine.inputs.reference = reference_file
      coreg_wo_affine.inputs.out_matrix = affine_mat
      coreg_wo_affine.inputs.cost = "crU"
      coreg_wo_affine.inputs.two_pass = True
      coreg_wo_affine.inputs.verbose = True
      coreg_wo_affine.inputs.out_file = output_file1
      coreg_wo_affine.inputs.out_param_file = "params.1D"
      coreg_wo_affine.run()

      print(f"[OK] Affine estimated and saved → {affine_mat}")
      print(f"[OK] Coregistered image (step 1) → {output_file1}")

      results["step1"] = output_file1

  # -------- STEP 2: Apply affine --------
  if apply_affine:
      if output_file2 is None:
          raise ValueError("output_file2 must be provided when apply_affine=True")
      if input_file2 is None:
          raise ValueError("input_file2 must be provided when apply_affine=True")


      coreg_with_affine = afni.Allineate()
      coreg_with_affine.inputs.in_file = input_file2
      coreg_with_affine.inputs.reference = reference_file
      coreg_with_affine.inputs.in_matrix = affine_mat
      coreg_with_affine.inputs.master = reference_file
      coreg_with_affine.inputs.verbose = True
      coreg_with_affine.inputs.final_interpolation = "linear"
      coreg_with_affine.inputs.out_file = output_file2
      coreg_with_affine.run()

      print_statement(f"[OK] Affine applied → {output_file2}", bcolors.OKGREEN)
      results["step2"] = output_file2

  return results
def roi_analysis(func_in_file, roi_file, n_vols, tr, base_start, base_end, sig_start, sig_end):
    print_statement(f"Extracting time course for ROI: {roi_file}", bcolors.NOTIFICATION)
    output_file = f"time_course_{roi_file.replace('.nii.gz', '.txt')}"
    time_course_extraction(roi_file, func_in_file, output_file)
    print_statement(f"[OK] Time course saved → {output_file}", bcolors.OKGREEN)

    #Creating Percent Signal Change graphs for each ROI
    id_arr = list(range(0, n_vols, tr))
    time_series = np.loadtxt(output_file)
    # baseline = np.mean(time_series[base_start:base_end])
    baseline = np.mean(time_series[base_start:base_end])
    psc = ((time_series - baseline) / baseline) * 100
    mean_signal = np.mean(psc[sig_start:sig_end])
    print_statement(f"[OK] Percent Signal Change calculated for ROI: {roi_file}", bcolors.OKGREEN)
    print("Time Series is:", psc)
    np.savetxt(f"PSC_time_series_{roi_file.replace('.nii.gz', '.txt')}", psc)
    plt.figure(figsize=(10, 5))
    plt.plot(id_arr, psc, label='Percent Signal Change')
    plt.axvspan(base_start, base_end, color='green', alpha=0.3, label='Baseline Period')
    plt.axvspan(sig_start, sig_end, color='blue', alpha=0.3, label='Signal Period')
    plt.title(f'Percent Signal Change Time Series for {roi_file}')
    plt.xlabel('Time Points (Volumes)')
    plt.ylabel('MRI Signal Change (%)')
    plt.set_ylim = (-2, 10)
    plt.legend()
    plt.tight_layout()
    graph_file = f"PSC_Time_Series_{roi_file.replace('.nii.gz', '.svg')}"
    plt.savefig(graph_file, dpi=1200)
    print_statement(f"[OK] Percent Signal Change graph saved → {graph_file}", bcolors.OKGREEN)   

    return mean_signal
def run_cmd(cmd):
    """Run shell command and return stdout."""
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
    return result.stdout.strip()
def create_intensity_mask(input_img, output_mask, low_percentile, high_percentile):
    """
    Create binary mask keeping voxels between given percentiles.

    Parameters
    ----------
    input_img : str
        Path to 3D NIfTI image
    output_mask : str
        Output mask filename
    low_percentile : float
        Lower percentile (e.g., 20)
    high_percentile : float
        Upper percentile (e.g., 80)
    """

    # -------------------------
    # Validate percentiles
    # -------------------------
    if not (0 <= low_percentile < high_percentile <= 100):
        raise ValueError("Percentiles must satisfy: 0 <= low < high <= 100")

    # Get percentile thresholds
    p_low = float(run_cmd(["fslstats", input_img, "-l", "0.0001", "-P", str(low_percentile)]))
    p_high = float(run_cmd(["fslstats", input_img, "-l", "0.0001", "-P", str(high_percentile)]))


    # Create binary mask
    subprocess.run(["fslmaths", input_img, "-thr", str(p_low), "-uthr", str(p_high), "-bin", output_mask], check=True)

    print(f"Mask created: {output_mask}")
    print(f"Thresholds used: {p_low:.3f} – {p_high:.3f}")
def make_3x3_roi_same_slice(ref_nii_path, center_phase, center_slice, center_read, out_path):
    """
    Creates a 3x3 ROI (9 voxels) centered at (phase, slice, read),
    keeping slice fixed, expanding +/-1 in phase and read.

    Axis convention: data[phase, slice, read]
    """
    ref_img = nib.load(ref_nii_path)
    shape = ref_img.shape

    roi = np.zeros(shape, dtype=np.uint8)

    p0, s0, r0 = int(center_phase), int(center_slice), int(center_read)

    # Clip to bounds (if center is near edges, ROI may contain < 9 voxels)
    p1 = max(p0 - 1, 0)
    p2 = min(p0 + 1, shape[0] - 1)
    r1 = max(r0 - 1, 0)
    r2 = min(r0 + 1, shape[2] - 1)

    if not (0 <= s0 < shape[1]):
        raise ValueError(f"center_slice {s0} out of bounds for shape {shape}")

    roi[p1:p2+1, s0, r1:r2+1] = 1

    out_img = nib.Nifti1Image(roi, ref_img.affine, header=ref_img.header.copy())
    out_img.set_data_dtype(np.uint8)
    nib.save(out_img, out_path)

    return out_path
def save_roi_from_max(scm_path, hemisphere_mask_path, res):
    # Decide output name from hemisphere mask filename
    mask_name = os.path.basename(hemisphere_mask_path).lower()
    out_name = "roi_left.nii.gz" if "left" in mask_name else "roi_right.nii.gz"

    out_path = os.path.join(os.path.dirname(hemisphere_mask_path), out_name)

    make_3x3_roi_same_slice(ref_nii_path=scm_path, center_phase=res["phase"], center_slice=res["slice"], center_read=res["read"], out_path=out_path)
    return out_path
def max_voxel_in_mask(scm_path, mask_path, slice_pick=None, slice_pad=1):
    """
    
    Finds the voxel with highest SCM intensity inside a binary mask.

    Axis convention assumed (same as your code):
      scm_data[phase, slice, read]

    slice_pick:
      - None -> search all slices in the mask
      - int  -> search only slice_pick ± slice_pad
    """
    scm_img  = nib.load(scm_path)
    scm_data = scm_img.get_fdata()

    mask = nib.load(mask_path).get_fdata() > 0

    if mask.shape != scm_data.shape:
        raise ValueError(f"Mask shape {mask.shape} != SCM shape {scm_data.shape}")

    # Optional: restrict to user-chosen slice ± pad
    if slice_pick is not None:
        sp = int(slice_pick)
        pad = int(slice_pad)

        coords = np.argwhere(mask)  # (phase, slice, read)
        keep = (coords[:, 1] >= sp - pad) & (coords[:, 1] <= sp + pad)

        coords = coords[keep]
        if coords.size == 0:
            return None

        vals = scm_data[coords[:, 0], coords[:, 1], coords[:, 2]]
        idx  = int(np.nanargmax(vals))
        phase, slice_, read = coords[idx]
        value = float(vals[idx])

    else:
        coords = np.argwhere(mask)          # (phase, slice, read)
        vals   = scm_data[mask]
        idx    = int(np.nanargmax(vals))
        phase, slice_, read = coords[idx]
        value  = float(vals[idx])

    return {"read": int(read), "phase": int(phase), "slice": int(slice_), "value": value}


# Helper functions, compulsory to run
def time_course_extraction(roi_file, func_file, output_file):
   
    ts = fsl.ImageMeants()
    ts.inputs.in_file = func_file
    ts.inputs.mask = roi_file
    ts.inputs.out_file = output_file

    ts.run()
def func_param_extract(scan_dir, export_env=True):

    scan_dir = Path(scan_dir)
    acqp_file = scan_dir / "acqp"
    method_file = scan_dir / "method"
    

    if not acqp_file.exists() or not method_file.exists():
        raise FileNotFoundError("acqp or method file not found")

    # -----------------------------
    # Read files
    # -----------------------------
    acqp_text = acqp_file.read_text()
    method_text = method_file.read_text()

    # -----------------------------
    # Sequence name (ACQ_protocol_name)
    # -----------------------------
    seq_match = re.search(
        r"ACQ_protocol_name=\(\s*64\s*\)\s*\n\s*<([^>]+)>",
        acqp_text
    )
    SequenceName = seq_match.group(1) if seq_match else None

    # -----------------------------
    # Extract numeric parameters
    # -----------------------------
    def get_value(pattern, text, cast=int):
        m = re.search(pattern, text)
        return cast(m.group(1)) if m else None

    NoOfRepetitions = get_value(r"##\$PVM_NRepetitions=\s*(\d+)", method_text)
    TotalScanTime = get_value(r"##\$PVM_ScanTime=\s*(\d+)", method_text)

    Baseline_TRs = get_value(r"PreBaselineNum=\s*(\d+)", method_text)
    StimOn_TRs = get_value(r"StimNum=\s*(\d+)", method_text)
    StimOff_TRs = get_value(r"InterStimNum=\s*(\d+)", method_text)
    NoOfEpochs = get_value(r"NEpochs=\s*(\d+)", method_text)

    # -----------------------------
    # Derived values
    # -----------------------------
    VolTR_msec = None
    VolTR = None
    MiddleVolume = None

    if NoOfRepetitions and TotalScanTime:
        VolTR_msec = TotalScanTime / NoOfRepetitions
        VolTR = VolTR_msec / 1000
        MiddleVolume = NoOfRepetitions / 2

    # -----------------------------
    # Pack results
    # -----------------------------
    params = {
        "SequenceName": SequenceName,
        "NoOfRepetitions": NoOfRepetitions,
        "TotalScanTime": TotalScanTime,
        "VolTR_msec": VolTR_msec,
        "VolTR": VolTR,
        "Baseline_TRs": Baseline_TRs,
        "StimOn_TRs": StimOn_TRs,
        "StimOff_TRs": StimOff_TRs,
        "NoOfEpochs": NoOfEpochs,
        "MiddleVolume": MiddleVolume,
    }

    # -----------------------------
    # Export to environment (optional)
    # -----------------------------
    if export_env:
        for k, v in params.items():
            if v is not None:
                os.environ[k] = str(v)

    return params
def extract_subject_id(scan_dir):
    subject_file = Path(scan_dir) / "subject"

    if not subject_file.exists():
        raise FileNotFoundError(f"'subject' file not found in {scan_dir}")

    with subject_file.open("r") as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        if line.strip().startswith("##$SUBJECT_id"):
            # The value is expected on the next line
            value_line = lines[i + 1].strip()
            return value_line.strip("<>")

    raise ValueError("##$SUBJECT_id not found in subject file")
def print_header(message, color):
    line = "*" * 134   # same width everywhere
    width = len(line)

    print()
    print(f"{color}{line}{bcolors.ENDC}")
    print(f"{color}{message.center(width)}{bcolors.ENDC}")
    print(f"{color}{line}{bcolors.ENDC}")
    print()
def print_statement(message, color):
    print(f"{color}{message}{bcolors.ENDC}")
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    NOTIFICATION = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
def view_images(input_image, cmap="gray"):
    img = nib.load(input_image)
    data = img.get_fdata()

    # Handle 4D vs 3D safely
    if data.ndim == 4:
        data = data[..., 0]   # take first volume
    elif data.ndim != 3:
        raise ValueError(f"Unsupported data shape: {data.shape}")

    ny = data.shape[1]
    cols = 8
    rows = int(np.ceil(ny / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    axes = np.atleast_1d(axes).flatten()

    for i, ax in enumerate(axes):
        if i < ny:
            ax.imshow(data[:, i, :].T, cmap=cmap, origin="lower")
            ax.set_title(f"y={i}", fontsize=14)
        ax.axis("off")

    plt.tight_layout()
    plt.show()
def matching_nifti_files(directory, ref_shape):
    matches = []

    ref_xyz = tuple(ref_shape[:3])  # (64,16,64)
    print(f"Reference spatial shape: {ref_xyz}")

    for f in sorted(os.listdir(directory)):
        if f.endswith((".nii", ".nii.gz")):
            try:
                img = nib.load(os.path.join(directory, f))
                img_xyz = tuple(img.shape[:3])

                if img_xyz == ref_xyz:
                    matches.append(f)

            except Exception as e:
                print(f"Skipping {f}: {e}")

    
    return matches
def roi_analysis(func_in_file, roi_file, n_vols, tr, base_start, base_end, sig_start, sig_end):
    print_statement(f"Extracting time course for ROI: {roi_file}", bcolors.NOTIFICATION)
    output_file = f"time_course_{roi_file.replace('.nii.gz', '.txt')}"
    time_course_extraction(roi_file, func_in_file, output_file)
    print_statement(f"[OK] Time course saved → {output_file}", bcolors.OKGREEN)

    #Creating Percent Signal Change graphs for each ROI
    id_arr = list(range(0, n_vols, tr))
    time_series = np.loadtxt(output_file)
    # baseline = np.mean(time_series[base_start:base_end])
    baseline = np.mean(time_series[base_start:base_end])
    psc = ((time_series - baseline) / baseline) * 100
    mean_signal = np.mean(psc[sig_start:sig_end])
    print_statement(f"[OK] Percent Signal Change calculated for ROI: {roi_file}", bcolors.OKGREEN)
    print("Time Series is:", psc)
    np.savetxt(f"PSC_time_series_{roi_file.replace('.nii.gz', '.txt')}", psc)
    plt.figure(figsize=(10, 5))
    plt.plot(id_arr, psc, label='Percent Signal Change')
    plt.axvspan(base_start, base_end, color='green', alpha=0.3, label='Baseline Period')
    plt.axvspan(sig_start, sig_end, color='blue', alpha=0.3, label='Signal Period')
    plt.title(f'Percent Signal Change Time Series for {roi_file}')
    plt.xlabel('Time Points (Volumes)')
    plt.ylabel('MRI Signal Change (%)')
    plt.set_ylim = (-2, 10)
    plt.legend()
    plt.tight_layout()
    graph_file = f"PSC_Time_Series_{roi_file.replace('.nii.gz', '.svg')}"
    plt.savefig(graph_file, dpi=1200)
    print_statement(f"[OK] Percent Signal Change graph saved → {graph_file}", bcolors.OKGREEN)   

    return mean_signal
def int_widget(value, desc):
    return widgets.IntText(
        value=value,
        description=desc,
        layout=INPUT_LAYOUT,
        style={'description_width': DESC_WIDTH}
    )
def print_selected_pipeline_steps():
    print("\nPIPELINE STEP SELECTION SUMMARY")
    print("-" * 40)

    for step in PIPELINE_STEPS:
        key = step["key"]
        name = step["name"]

        if checkboxes[key].value:
            print(f"[✓] {name}")
        else:
            print(f"[ ] {name}")

    print("-" * 40)
def shrink_mask_xz_linewise(in_mask, out_mask, trim_x=3, trim_z=3):
    img = nib.load(in_mask)
    data = img.get_fdata()

    if data.ndim != 3:
        raise ValueError("Input mask must be 3D")

    X, Y, Z = data.shape
    mask = data.copy()

    # Temporary mask after X-shrink
    tmp_mask = np.zeros_like(mask, dtype=mask.dtype)
    final_mask = np.zeros_like(mask, dtype=mask.dtype)

    # ==========================================================
    # STEP 1 — Shrink along X for every (Y, Z)
    # ==========================================================
    for y in range(Y):
        for z in range(Z):
            line = mask[:, y, z]
            idx = np.where(line > 0)[0]

            if idx.size < 2 * trim_x + 1:
                continue

            x_min = idx.min() + trim_x
            x_max = idx.max() - trim_x

            tmp_mask[x_min:x_max + 1, y, z] = mask[x_min:x_max + 1, y, z]

    # ==========================================================
    # STEP 2 — Shrink along Z for every (Y, X)
    # ==========================================================
    for y in range(Y):
        for x in range(X):
            line = tmp_mask[x, y, :]
            idx = np.where(line > 0)[0]

            if idx.size < 2 * trim_z + 1:
                continue

            z_min = idx.min() + trim_z
            z_max = idx.max() - trim_z

            final_mask[x, y, z_min:z_max + 1] = \
                tmp_mask[x, y, z_min:z_max + 1]

    nib.save(
        nib.Nifti1Image(final_mask, img.affine, img.header),
        out_mask
    )

    print(f"[OK] Line-wise shrunk mask saved → {out_mask}")
def sanitize(value, na_text="NA"):
    """
    Convert undefined / empty values to 'NA'
    """
    if value is None:
        return na_text
    if isinstance(value, str) and value.strip() == "":
        return na_text
    return value
def safe_get(varname):
    """
    Safely get a variable that may not exist.
    """
    return locals().get(varname, globals().get(varname, None))
def validate_analysis_inputs(fields):
    """
    Logical validation only.
    Missing values are allowed and stored as NA.
    """

    if fields["temporal_res"] not in (None, "NA"):
        if fields["temporal_res"] <= 0:
            raise RuntimeError("Invalid temporal resolution (TR must be > 0)")

    if fields["window_duration"] not in (None, "NA"):
        if fields["window_duration"] <= 0:
            raise RuntimeError("Invalid window duration")

    if (
        fields["start_idx_correlation"] not in (None, "NA")
        and fields["end_idx_correlation"] not in (None, "NA")
    ):
        if fields["start_idx_correlation"] >= fields["end_idx_correlation"]:
            raise RuntimeError("Invalid correlation window indices")
def is_step_enabled(step_key):
    return checkboxes.get(step_key, None) and checkboxes[step_key].value


In [ ]:
PIPELINE_STEPS = [
    {"name": "Bruker → NIfTI", "key": "bruker_to_nifti", "func": bruker_to_nifti, "inputs": ["in_path", "scan_number"], "produces": "func_file"},
    {"name": "Motion Correction", "key": "motion_correction", "func": motion_correction, "inputs": ["reference_vol", "func_file", "output_prefix"], "produces": "func_file"},
    {"name": "Temporal Smoothing (MovAvg)", "key": "smooth_movavg", "func": smooth_movavg, "inputs": ["func_file", "out_file", "win_sec_duration", "tr"], "produces": "func_file"},
    {"name": "Spatial Smoothing", "key": "spatial_smoothing", "func": spatial_smoothing, "inputs": ["func_file", "out_file", "fwhm"], "produces": "func_file"},
    {"name": "Compute Mean Baseline", "key": "compute_mean_range", "func": compute_mean_range, "inputs": ["func_file", "prefix", "start_idx", "end_idx"], "produces": "baseline_file"},
    {"name": "Masking", "key": "masking_file", "func": masking_file, "inputs": ["func_file", "mask_file", "out_file"], "produces": "func_file"},
    {"name": "Temporal SNR Estimation", "key": "tSNR", "func": tSNR, "inputs": ["func_file", "out_file", "reference_vol", "size"], "produces": "tsnr_file"},
    {"name": "Signal Change Map", "key": "signal_change_map", "func": signal_change_map, "inputs": ["func_file", "baseline_file", "out_file"], "produces": "psc_file"},
    {"name": "ROI Analysis", "key": "roi_analysis", "func": roi_analysis, "inputs": ["func_in_file", "roi_file", "n_vols", "tr", "base_start", "base_end", "sig_start", "sig_end"], "produces": "roi_graphs"},
    {"name": "Coregistration", "key": "coregistration_afni", "func": coregistration_afni, "inputs": ["input_file1", "input_file2", "reference_file"], "produces": "roi_graphs"}]

checkboxes = {}

for step in PIPELINE_STEPS:
    cb = widgets.Checkbox(value=True, description=step["name"], indent=False)
    checkboxes[step["key"]] = cb

box = widgets.Box(list(checkboxes.values()), layout=widgets.Layout(display="flex", flex_flow="row wrap", align_items="flex-start", gap="10px"))
display(box)

In [ ]:
## Make sure what functionas are selected to run in the pipeline
print_selected_pipeline_steps()

## Choosing the raw data that needs to be analysed
root_location = "/Volumes/Extreme_Pro/fMRI"
selected_folder = FileChooser(root_location, layout=widgets.Layout(width='1080px'))
selected_folder.show_only_dirs = True
display(selected_folder)

In [ ]:
in_path = Path(selected_folder.selected_path)
print(f"Our Raw Data is located in the path {in_path}")
subject_id = extract_subject_id(in_path)

# ---------- Common styles ----------
INPUT_LAYOUT = widgets.Layout(width="280px", flex="0 0 auto")
ROW_LAYOUT = widgets.Layout(width="100%", justify_content="space-between", align_items="center", padding="12px", border="3px solid #444", border_radius="10px", background_color="#1e1e1e")
DESC_WIDTH = "220px"

# ---------- Widgets (KEEP REFERENCES) ----------
func_scan_number_entered   = int_widget(10, "Functional Scan:")
struct_scan_number_entered = int_widget(10, "Structural Scan:")
win_entered         = int_widget(200, "Window Duration (in vols):")
temp_smoothing_window = int_widget(60, "Temporal Smoothing (in vols):")

row = widgets.HBox([func_scan_number_entered, struct_scan_number_entered, win_entered, temp_smoothing_window], layout=ROW_LAYOUT)
display(row)


In [ ]:
func_scan_number = str(func_scan_number_entered.value)
struct_scan_number = str(struct_scan_number_entered.value)
win = str(win_entered.value)

print_statement(f"Subject under analysis is {subject_id} for Functional Scan Number {func_scan_number} and Structural Scan Number {struct_scan_number} with window size {win}", bcolors.NOTIFICATION)

# # Parameters extraction for structural and functional files

params_struct = func_param_extract(Path(in_path) / struct_scan_number,export_env=True)
sequence_name_struct = params_struct["SequenceName"]

params_func = func_param_extract(Path(in_path) / func_scan_number,export_env=True)
sequence_name_func = params_func["SequenceName"]


# Setting up path for storing analysed data overall, strucutral data, functional data and processed functional data
# Analysed data overall
parts = list(in_path.parts)
parts[parts.index("RawData")] = "AnalysedData"
analysed_base = Path(*parts).parent
analysed_path = analysed_base / subject_id

# Structural and Functional data
analysed_struct_dir = Path(analysed_path) / f"{struct_scan_number}{sequence_name_struct}"
analysed_func_dir = Path(analysed_path) / f"{func_scan_number}{sequence_name_func}"

# processed functional data
timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")
user = getpass.getuser()
folder_created = f"{timestamp}_{user}"
analysis_dir = Path(analysed_path) / f"{func_scan_number}{sequence_name_func}" / folder_created
analysis_dir.mkdir(parents=True, exist_ok=False)  # fail loudly if it already exists

# src_dir = Path.cwd()

# path for raw strucutral and functional data 
path_raw_struct = os.path.join(in_path, struct_scan_number)
path_raw_func = os.path.join(in_path, func_scan_number)

# Display of variables and paths in tabular form
var_vals = [
            ['Location of all Raw Data', 'root_location', root_location],
            ['Location of Raw Data to be analysed', 'in_path', in_path],
            ['Location of Raw Strcutural Data', 'path_raw_struct', path_raw_struct],
            ['Location of Raw Functional Data', 'path_raw_func', path_raw_func],
            ['Location of analysed structural data', 'analysed_struct_dir', analysed_struct_dir],
            ['Location of analysed functional data', 'analysed_func_dir', analysed_func_dir],
            ['Lcoation of final processed functional dta', 'analysis_dir', analysis_dir]
            ]

# pd.set_option('display.max_colwidth', 500)  # or 199
df = pd.DataFrame(var_vals, columns=['Purpose', 'Variable Name', 'Value'], index=['a', 'b', 'c', 'd', 'e', 'f', 'g'])
print(tabulate(df, headers='keys', tablefmt='psql'))


print_statement("For your information: all generated files after processing stored in the directory will be utilising the below nomenclature:", bcolors.NOTIFICATION)

print(f"After applying motion correction, file will be saved as mc_input_file")
print(f"After applying temporal smoothing, file will be saved as ts_input_file")
print(f"After applying spatial smoothing, file will be saved as sm_input_file")
print(f"After applying temporal smoothing, file will be saved as ts_input_file")
print(f"After applying temporal SNR, file will be saved as tsnr_input_file")

# Converting Functional Data into NIFTI and Processing Functional Data

In [ ]:
print_statement(f"Analysed Data Path: {analysed_func_dir}", bcolors.NOTIFICATION)
analysed_func_dir.mkdir(parents=True, exist_ok=True)
os.chdir(analysed_func_dir)

if is_step_enabled("bruker_to_nifti"):
    print_header("Converting Bruker to NIFTI: Functional Data", bcolors.HEADER)

    nifti_file = analysed_func_dir / "func.nii.gz"

    if nifti_file.exists():
        print_statement("NIfTI file already exists. Skipping conversion.", bcolors.OKGREEN)
    else:
        bruker_to_nifti(in_path, func_scan_number, "func.nii.gz")

    input_name_for_next_step = "func"
    
else:
    print_statement("⏭️ Data Conversion Step not executed, May cause trouble in running further analysis", bcolors.NOTIFICATION)

input_name_for_next_step = "func"

# Generating Masks and Removing Border voxels from the handdrawn masks

In [ ]:
tr = int(params_func["VolTR"])
n_vols = params_func["NoOfRepetitions"]
middle_vol = str(int(n_vols / 2))

extract_middle_volume(f"{input_name_for_next_step}.nii.gz", int(middle_vol), "middle_vol.nii.gz", 1)

#Creating a mask to be applied on functional data using the mean baseline image

if is_step_enabled("masking_file"):
    
    mask_file= "mask_mean_mc_func.nii.gz"
    mask_file_cannulas= "mask_mean_mc_func_cannulas.nii.gz"
    
    mask_file_shrunk = os.path.join(analysed_func_dir, "mask_mean_mc_func_cleaned.nii.gz")
    mask_file_cannulas_shrunk = os.path.join(analysed_func_dir, "mask_mean_mc_func_cannulas_cleaned.nii.gz")
    

    if os.path.exists(mask_file):
        print(f"{bcolors.OKGREEN}Mask Image ({mask_file}) exists.{bcolors.ENDC}")
    else:
        print(f"{bcolors.FAIL}Mask Image does not exist. Please create the mask and save it as mask_mean_mc_func.nii.gz{bcolors.ENDC}")
        subprocess.run(["fsleyes", "middle_vol.nii.gz"])

    if os.path.exists(mask_file_cannulas):
        print(f"{bcolors.OKGREEN}Mask Image including cannulas ({mask_file_cannulas}) exist.{bcolors.ENDC}")
    else:
        print(f"{bcolors.FAIL}Mask Image does not exist. Please create the mask that also includes cannulas and save it as mask_mean_mc_func_cannulas.nii.gz.{bcolors.ENDC}")
        shutil.copyfile("mask_mean_mc_func.nii.gz", "mask_mean_mc_func_cannulas.nii.gz")
        subprocess.run(["fsleyes", "middle_vol.nii.gz" , "mask_mean_mc_func_cannulas.nii.gz"])
    
    shrink_mask_xz_linewise(in_mask=mask_file, out_mask=mask_file_shrunk, trim_x=2, trim_z=2)
    shrink_mask_xz_linewise(in_mask=mask_file_cannulas, out_mask=mask_file_cannulas_shrunk, trim_x=2, trim_z=2)

else:
    print_statement("⏭️ Masks not generated, will not get cleaned data in further pipeline", bcolors.NOTIFICATION)
  




# Applying Motion Correction in Raw Functional Data and Plotting Motion Parameters

In [ ]:
if is_step_enabled("motion_correction"):
    print_header("Applying Motion Correction on raw functional data and plotting motion parameters", bcolors.HEADER)

    if os.path.exists("mc_func.nii.gz"):
        print(f"{bcolors.OKGREEN}Motion Corrected functional data exists. Skipping motion correction.{bcolors.ENDC}")
    else:
        motion_correction("middle_vol.nii.gz", f"{input_name_for_next_step}.nii.gz", output_prefix=f"mc_{input_name_for_next_step}")
    
        
    #Display of the motion corrected image and motion corrected graphs
    input_name_for_next_step = f"mc_{input_name_for_next_step}"
    view_images("mc_func.nii.gz", cmap="gray")
    plot_motion_parameters("motion.1D")    
else:
    print_statement("⏭️ Motion Correction not executed", bcolors.NOTIFICATION)


# Move into further subdirectory within Analysed Data Folder

In [ ]:
print_statement(f"Analysed Data Path Final: {analysis_dir}", bcolors.NOTIFICATION)
analysis_dir.mkdir(parents=True, exist_ok=True)
os.chdir(analysis_dir)

mask_file = os.path.join(analysed_func_dir, "mask_mean_mc_func.nii.gz")
mask_file_cannulas = os.path.join(analysed_func_dir, "mask_mean_mc_func_cannulas.nii.gz")

shutil.copyfile(f"../{input_name_for_next_step}.nii.gz", f"{input_name_for_next_step}.nii.gz")

# Selecting Start indices for both: Baseline and Signal

In [ ]:
# Opening fsleyes to view the temporally smoothed motion corrected functional data
print_statement("Choose your baseline and signal volumes from the temporally smoothed motion corrected functional data.", bcolors.NOTIFICATION)
subprocess.run(["fsleyes", f"{input_name_for_next_step}.nii.gz"])

# ---------- Common styling ----------
input_layout = widgets.Layout(width="280px", flex="0 0 auto")
label_style = {'description_width': '410px'}

# ---------- Widgets ----------
base_start_idx = widgets.IntText(value=400, description="Baseline Start Index:", layout=input_layout, style=label_style)
sig_start_idx = widgets.IntText(value=10, description="Signal Start Index:", layout=input_layout, style=label_style)

for w in [base_start_idx, sig_start_idx]:
    w.style.description_width = "180px"

# ---------- Container ----------
row = widgets.HBox([base_start_idx, sig_start_idx],
    layout=widgets.Layout(width="40%", justify_content="space-between", align_items="center", padding="12px", border="3px solid #444", border_radius="10px", background_color="#1e1e1e"))

row.layout.background_color = "#1e1e1e"
row.layout.border = "3px solid #444"

display(row)

In [ ]:
base_start = int(base_start_idx.value)
sig_start  = int(sig_start_idx.value)
base_end   = base_start + int(win_entered.value)
sig_end   = sig_start + int(win_entered.value)

print_statement(f"Your baseline window selected is: {base_start}_to_{base_end}", bcolors.OKGREEN)
print_statement(f"Your signal window selected is: {sig_start}_to_{sig_end}", bcolors.OKGREEN)

# Apply Spatial Smoothing and Generating Signal Change Map

In [ ]:
input_name_for_next_step = "mc_func"

In [ ]:
# working_path = Path("/Volumes/Extreme_Pro/fMRI/AnalysedData/Project_SeroAVATar_NJ_KR/Project2_Sero_AAV/RGRO_260128_0224_RN_SD_393/20functionalEPI")
# cleaned_file_path = Path("/Volumes/Extreme_Pro/fMRI/AnalysedData/Project_SeroAVATar_NJ_KR/Project2_Sero_AAV/RGRO_260128_0224_RN_SD_393/20functionalEPI/2026_02_10_093806_njain")

motion_corrected_file = "mc_func.nii.gz"

print(f"{input_name_for_next_step}.nii.gz")

## Currently based on the previous results, one eee
if is_step_enabled("spatial_smoothing"):
    print_header("Applying Spatial Smoothing on motion corrected functional data", bcolors.HEADER)
    spatial_smoothing(f"{input_name_for_next_step}.nii.gz", f"sm_{input_name_for_next_step}.nii.gz", 0.297)
    input_name_for_next_step = f"sm_{input_name_for_next_step}"
else: 
    print_statement("⏭️ Spatial Smoothing not executed, will not get smoothed data in further pipeline", bcolors.NOTIFICATION)
    
compute_mean_range(input_file=f"{input_name_for_next_step}.nii.gz", prefix=f"mean_baseline_image_{base_start}_to_{base_end}.nii.gz", start_idx=base_start, end_idx=base_end)
compute_mean_range(input_file=f"{input_name_for_next_step}.nii.gz", prefix=f"mean_signal_image_{sig_start}_to_{sig_end}.nii.gz", start_idx=sig_start, end_idx=sig_end)

signal_change_map(f"mean_signal_image_{sig_start}_to_{sig_end}.nii.gz", f"mean_baseline_image_{base_start}_to_{base_end}.nii.gz", f"scm_from_sm_{input_name_for_next_step}.nii.gz") # Creating SCM image
signal_change_map(f"{input_name_for_next_step}.nii.gz", f"mean_baseline_image_{base_start}_to_{base_end}.nii.gz", f"scm_timeseries_from_sm_{input_name_for_next_step}.nii.gz") # Creating Signal Change Time Series


compute_mean_range(motion_corrected_file, "mean_mc_func.nii.gz", 200, 500)
masking_file("mean_mc_func.nii.gz", mask_file_shrunk, "cleaned_shrunk_mean_mc_func.nii.gz")
masking_file("mean_mc_func.nii.gz", mask_file, "cleaned_mean_mc_func.nii.gz")

# Create intensity mask based for the functional image
create_intensity_mask("cleaned_shrunk_mean_mc_func.nii.gz", "mask_thresholded.nii.gz", 15, 80)
final_mask_used = os.path.join(analysis_dir, "mask_thresholded.nii.gz")


if is_step_enabled("masking_file"):
    print_header("Applying Mask on motion corrected functional data", bcolors.HEADER)
    masking_file(f"scm_from_sm_{input_name_for_next_step}.nii.gz", final_mask_used, f"clean_thresh_scm_from_sm_{input_name_for_next_step}.nii.gz")
    masking_file(f"scm_from_sm_{input_name_for_next_step}.nii.gz", mask_file_shrunk, f"clean_shrunk_scm_from_sm_{input_name_for_next_step}.nii.gz")
    masking_file(f"scm_from_sm_{input_name_for_next_step}.nii.gz", mask_file, f"cleaned_scm_from_sm_{input_name_for_next_step}.nii.gz")

    scm = f"cleaned_scm_from_sm_{input_name_for_next_step}.nii.gz"
else:
    print_statement("⏭️ Masking not executed, will not get cleaned data in further pipeline", bcolors.NOTIFICATION)


input_file_for_coreg = analysis_dir / "cleaned_mean_mc_func.nii.gz"
scm_for_coreg = analysis_dir / f"cleaned_scm_from_sm_{input_name_for_next_step}.nii.gz"

# Converting Structural Data into NIFTI and Processing it

In [ ]:
print_statement(f"Analysed Data Path: {analysed_struct_dir}", bcolors.NOTIFICATION)
analysed_struct_dir.mkdir(parents=True, exist_ok=True)
os.chdir(analysed_struct_dir)

if is_step_enabled("bruker_to_nifti"):
    print_header("Converting Bruker to NIFTI: Structural Data", bcolors.HEADER)

    nifti_file = analysed_struct_dir / "struct.nii.gz"
    if nifti_file.exists():
        print_statement("NIfTI file already exists. Skipping conversion.", bcolors.OKGREEN)
    else:
        bruker_to_nifti(in_path, struct_scan_number, "struct.nii.gz")

else:
    print_statement("⏭️ Data Conversion Step not executed, May cause trouble in running further analysis", bcolors.NOTIFICATION)


#Cleaning the structural image by masking it with a manually created mask

if is_step_enabled("masking_file"):
    print_statement("Cleaning the structural image by manually creating mask", bcolors.NOTIFICATION)
    
    if os.path.exists("cleaned_struct.nii.gz"):
        print_statement("Structural Image for Coregistration exists.", bcolors.OKGREEN)
    else:
        print_statement("Please create a mask for the structural image and save it as mask_struct.nii.gz", bcolors.NOTIFICATION)
        subprocess.run(["fsleyes", "struct.nii.gz", os.path.join(analysed_func_dir, "middle_vol.nii.gz")])
        masking_file("struct.nii.gz", "mask_struct.nii.gz", "cleaned_struct.nii.gz")

    structural_file_for_coregistration = os.path.join(analysed_struct_dir, "cleaned_struct.nii.gz")    
    view_images("cleaned_struct.nii.gz", cmap="gray")
    
    #Further shrinking the mask for structural image to get better coregistration results as structural image has high spatial resolution than functional image
    

else:
    print_statement("⏭️ Masks not generated, will not get cleaned data in further pipeline", bcolors.NOTIFICATION)    
    structural_file_for_coregistration = os.path.join(analysed_struct_dir, "struct.nii.gz")    
    view_images("struct.nii.gz", cmap="gray")



In [ ]:

print_statement(f"Analysed Data Path Final: {analysis_dir}", bcolors.NOTIFICATION)
analysis_dir.mkdir(parents=True, exist_ok=True)
os.chdir(analysis_dir)

coregistration_afni(input_file_for_coreg, scm_for_coreg, structural_file_for_coregistration, output_file1="mean_func_struct_aligned.nii.gz", output_file2="signal_change_map_coregistered_structural_space.nii.gz", estimate_affine=True, apply_affine=True, affine_mat="mean_func_struct_aligned.aff12.1D") # Estimating affnine and applying it to SCM


create_intensity_mask(structural_file_for_coregistration, "struct_mask_thresholded.nii.gz", 15, 80)
struct_final_mask_used = os.path.join(analysis_dir, "struct_mask_thresholded.nii.gz")
shrink_mask_xz_linewise(in_mask="struct_mask_thresholded.nii.gz", out_mask="struct_mask_thresholded_shrunk.nii.gz", trim_x=2, trim_z=2)  
masking_file("signal_change_map_coregistered_structural_space.nii.gz", "struct_mask_thresholded_shrunk.nii.gz", "cleaned_signal_change_map_coregistered_structural_space.nii.gz")


# ROI Analysis

In [ ]:
mask = nib.load(mask_file_shrunk).get_fdata().astype(bool)
func = nib.load("cleaned_mean_mc_func.nii.gz").get_fdata().astype(bool)


voxel_indices_func = np.column_stack(np.where(func))
voxel_indices_mask = np.column_stack(np.where(mask))


df_indices = pd.DataFrame(voxel_indices_mask)
# display(df_indices)

display(df_indices.columns)
# display(df_indices)

voxel_indices_func = np.column_stack(np.where(func))
voxel_indices_mask = np.column_stack(np.where(mask))

df_indices = pd.DataFrame(voxel_indices_mask)
df_indices.columns = ["read", "slice", "phase"]

display(df_indices.columns)
display(df_indices)

read  = df_indices["read"].to_numpy()
phase = df_indices["phase"].to_numpy()
slice = df_indices["slice"].to_numpy()

read_min = int(read.min())
read_max = int(read.max())

phase_min = int(phase.min())
phase_max = int(phase.max())

slice_min = int(slice.min())
slice_max = int(slice.max())


display(f"Range of mask in Read dim: {read_min} to {read_max}")
display(f"Range of mask in Phase dim: {phase_min} to {phase_max}")
display(f"Range of mask in Slice dim: {slice_min} to {slice_max}")

center = np.array([(read_min  + read_max)  // 2, (slice_min + slice_max) // 2, (phase_min + phase_max) // 2])

# find closest mask voxel
voxels = voxel_indices_mask
dists = np.linalg.norm(voxels - center, axis=1)
mid_val_read, mid_val_slice, mid_val_phase = voxels[np.argmin(dists)]

print(mid_val_read, mid_val_slice, mid_val_phase)


hemisphere_right = df_indices[(df_indices['read'] < read_max) & (df_indices['read'] > mid_val_read) & (df_indices['phase'] < phase_max) & (df_indices['phase'] > phase_min)]
hemisphere_left = df_indices[(df_indices['read'] > read_min) & (df_indices['read'] < mid_val_read) & (df_indices['phase'] < phase_max) & (df_indices['phase'] > phase_min)]

def create_binary_mask(coords, shape):
    binary_mask = np.zeros(shape, dtype=np.uint8)
    coords = np.array(coords, dtype=int)
    binary_mask[coords[:, 0], coords[:, 1], coords[:, 2]] = 1
    return binary_mask

mask_img = nib.load(mask_file_shrunk)
mask_shape = mask_img.shape
affine = mask_img.affine

mask_hemisphere_right = create_binary_mask(hemisphere_right, mask_shape)
mask_hemisphere_left = create_binary_mask(hemisphere_left, mask_shape)

nib.save(nib.Nifti1Image(mask_hemisphere_left, affine), "hemisphere_left.nii.gz")
nib.save(nib.Nifti1Image(mask_hemisphere_right, affine), "hemisphere_right.nii.gz")

In [ ]:
scm = f"cleaned_scm_from_sm_{input_name_for_next_step}.nii.gz"

In [ ]:
# Opening fsleyes to view the temporally smoothed motion corrected functional data

# ---------- Common styling ----------
input_layout = widgets.Layout(width="280px", flex="0 0 auto")
label_style = {'description_width': '410px'}

# ---------- Widgets ----------
right_slice = widgets.IntText(value=10, description="Right Hemisphere Slice:", layout=input_layout, style=label_style)
left_slice = widgets.IntText(value=10, description="Left Hemisphere Slice:", layout=input_layout, style=label_style)

for w in [right_slice, left_slice]:
    w.style.description_width = "180px"

# ---------- Container ----------
row = widgets.HBox([right_slice, left_slice],
    layout=widgets.Layout(width="40%", justify_content="space-between", align_items="center", padding="12px", border="3px solid #444", border_radius="10px", background_color="#1e1e1e"))

row.layout.background_color = "#1e1e1e"
row.layout.border = "3px solid #444"

display(row)

In [ ]:
right_slice_choosen = int(right_slice.value)
left_slice_choosen  = int(left_slice.value)

print_statement(f"Your choosen slice for right hemisphere is: {right_slice_choosen}", bcolors.OKGREEN)
print_statement(f"Your choosen slice for left hemisphere is: {left_slice_choosen}", bcolors.OKGREEN)

In [ ]:

scm_path = "clean_thresh_scm_from_sm_sm_mc_func.nii.gz"

res_left  = max_voxel_in_mask(scm_path, "hemisphere_left.nii.gz",  slice_pick=left_slice_choosen,  slice_pad=2)
left_roi = save_roi_from_max(scm_path="clean_thresh_scm_from_sm_sm_mc_func.nii.gz", hemisphere_mask_path="hemisphere_left.nii.gz", res=res_left)
roi_analysis(f"{input_name_for_next_step}.nii.gz", left_roi, n_vols, tr, base_start, base_end, sig_start, sig_end)
print("LEFT:", res_left)
print("Saved:", left_roi)

res_right = max_voxel_in_mask(scm_path, "hemisphere_right.nii.gz", slice_pick=right_slice_choosen, slice_pad=2)
right_roi = save_roi_from_max(scm_path="clean_thresh_scm_from_sm_sm_mc_func.nii.gz", hemisphere_mask_path="hemisphere_right.nii.gz", res=res_right)
roi_analysis(f"{input_name_for_next_step}.nii.gz", right_roi, n_vols, tr, base_start, base_end, sig_start, sig_end)
print("RIGHT:", res_right)
print("Saved:", right_roi)


